In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!ls -la /content/drive/MyDrive


total 2986951
-rw------- 1 root root 3053594823 Dec 12 09:41  archive.zip
drwx------ 2 root root       4096 Nov  5 07:32 'Colab Notebooks'
-rw------- 1 root root     386560 Oct 26 14:21  FMch1
-rw------- 1 root root    1665082 Nov 27 17:34  IMG-20251127-WA0020.jpeg
-rw------- 1 root root    1471124 Nov 27 17:34  IMG-20251127-WA0030.jpg
-rw------- 1 root root     172037 Dec  5 09:43  lesson5.pdf
-rw------- 1 root root     148643 Dec  5 09:45  lesson6.pdf
-rw------- 1 root root     568186 Dec 12 10:22  lesson7.pdf
-rw------- 1 root root     624235 Dec 12 10:21  lesson8.pdf


In [3]:
!mkdir -p /content/drive/MyDrive/dataset
!unzip -q /content/drive/MyDrive/archive.zip -d /content/drive/MyDrive/dataset/


In [5]:
!ls -la /content/drive/MyDrive/dataset/dataset


total 8
drwx------ 16 root root 4096 Dec 12 19:22 Test
drwx------ 20 root root 4096 Dec 12 19:30 Train


In [7]:
import os  # <-- needed

TRAIN_DIR = '/content/drive/MyDrive/dataset/dataset/Train'
TEST_DIR = '/content/drive/MyDrive/dataset/dataset/Test'

print("Train exists:", os.path.exists(TRAIN_DIR))
print("Test exists:", os.path.exists(TEST_DIR))



Train exists: True
Test exists: True


In [8]:
import os

common_classes = sorted([d for d in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, d))])
print("Detected classes:", common_classes)


Detected classes: ['freshapples', 'freshbanana', 'freshbittergroud', 'freshcapsicum', 'freshcucumber', 'freshokra', 'freshoranges', 'freshpotato', 'freshtomato', 'rottenapples', 'rottenbanana', 'rottenbittergroud', 'rottencapsicum', 'rottencucumber', 'rottenokra', 'rottenoranges', 'rottenpotato', 'rottentomato']


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_WIDTH = 150
IMG_HEIGHT = 150
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.20  # 80% training, 20% validation
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    classes=common_classes
)

validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    classes=common_classes
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    classes=common_classes
)

print("Generators created successfully.")
print("Class indices:", train_generator.class_indices)


Found 18901 images belonging to 18 classes.
Found 4718 images belonging to 18 classes.
Found 5490 images belonging to 18 classes.
Generators created successfully.
Class indices: {'freshapples': 0, 'freshbanana': 1, 'freshbittergroud': 2, 'freshcapsicum': 3, 'freshcucumber': 4, 'freshokra': 5, 'freshoranges': 6, 'freshpotato': 7, 'freshtomato': 8, 'rottenapples': 9, 'rottenbanana': 10, 'rottenbittergroud': 11, 'rottencapsicum': 12, 'rottencucumber': 13, 'rottenokra': 14, 'rottenoranges': 15, 'rottenpotato': 16, 'rottentomato': 17}


In [12]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

base_model = MobileNetV2(
    input_shape=(IMG_WIDTH, IMG_HEIGHT, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(len(common_classes), activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


/tmp/ipython-input-1942955404.py:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 5, 5, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 18)             │         4,626 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,590,546 (9.88 MB)

 Trainable params: 332,562 (1.27 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [13]:
EPOCHS = 5

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
591/591 ━━━━━━━━━━━━━━━━━━━━ 695s 1s/step - accuracy: 0.8039 - loss: 0.6740 - val_accuracy: 0.8730 - val_loss: 0.5472
Epoch 2/5
591/591 ━━━━━━━━━━━━━━━━━━━━ 680s 1s/step - accuracy: 0.9612 - loss: 0.1155 - val_accuracy: 0.8510 - val_loss: 0.6451
Epoch 3/5
591/591 ━━━━━━━━━━━━━━━━━━━━ 641s 1s/step - accuracy: 0.9701 - loss: 0.0797 - val_accuracy: 0.8423 - val_loss: 0.6912
Epoch 4/5
591/591 ━━━━━━━━━━━━━━━━━━━━ 635s 1s/step - accuracy: 0.9804 - loss: 0.0577 - val_accuracy: 0.8709 - val_loss: 0.5245
Epoch 5/5
591/591 ━━━━━━━━━━━━━━━━━━━━ 651s 1s/step - accuracy: 0.9828 - loss: 0.0468 - val_accuracy: 0.8709 - val_loss: 0.5796


In [14]:
model.save('/content/drive/MyDrive/food_freshness_model.h5')
print("Model saved to Drive successfully!")


Model saved to Drive successfully!


In [15]:
from tensorflow.keras.models import load_model

loaded_model = load_model('/content/drive/MyDrive/food_freshness_model.h5')
print("Model loaded successfully!")


Model loaded successfully!


In [16]:
import numpy as np
from PIL import Image
import gradio as gr

# Map class indices back to class names
index_to_class = {v: k for k, v in train_generator.class_indices.items()}

def predict_freshness(img):
    img = img.resize((IMG_WIDTH, IMG_HEIGHT))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    preds = loaded_model.predict(img_array)
    class_index = np.argmax(preds[0])
    class_name = index_to_class[class_index]
    confidence = float(np.max(preds[0]))

    return {class_name: confidence}


In [17]:
iface = gr.Interface(
    fn=predict_freshness,
    inputs=gr.Image(type="pil", label="Upload Food Image"),
    outputs=gr.Label(num_top_classes=1),
    title="Food Freshness Classifier",
    description="Upload an image to classify it as Fresh or Rotten."
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5c325cf5d87191be70.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
